In [25]:
# !pip install pandas


import pandas as pd
import numpy as np
import json

# load environment variables
# !pip install python-dotenv
# !pip install -U langchain-community
from dotenv import load_dotenv
load_dotenv()
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# !pip install langchain_pinecone
from langchain_pinecone import PineconeVectorStore, PineconeEmbeddings

from langchain.document_loaders import TextLoader

In [26]:
# read the data "01 課程大綱_202408201909.json" into panda file

with open('課程大綱.json') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# course = df['course']
course = list(data.values())[0]

course = pd.json_normalize(course)


In [27]:
print(course.columns)

Index(['開課學期', '開課學校', '課程狀態', '課程流水號', '課號', '課程識別碼', '班次', '課程名稱', '學分',
       '為必選修', '為預分配', '通識領域', '授課對象系所代碼', '授課對象系所名稱', '授課教師', '職稱', '所屬學院',
       '所屬系所', 'E-mail', '辦公室', '聯絡電話', '個人資訊', '個人網站', '密集課程週次', '上課星期',
       '上課節次', '上課地點', '加簽方式', '修課總人數', '本校人數', '外校人數', '外系人數', '領域專長代碼',
       '領域專長名稱', '為英語授課', 'NTU COOL', '核心能力與課程規劃關聯圖', '課程與基本素養關聯圖', '下載課程大綱檔案',
       '課程備註', '課程概述', '課程目標', '課程要求', '預計課後學習時數', 'OfficeHour需預約',
       'OfficeHour備註', 'OfficeHour星期', 'OfficeHour開始時間', 'OfficeHour結束時間',
       '指定閱讀', '參考書目'],
      dtype='object')


In [28]:
# print the szize of the data
print(course.shape)

(83547, 51)


In [29]:
#  filter 開課學期 = 113-1
course = course[course['開課學期'] == '113-1']


In [30]:
print(course.shape)

(17343, 51)


In [31]:
# 只保留
# "課程名稱"
# "課程流水號"
# "所屬系所"
# "授課教師"
# "上課星期"
# "上課節次"
# "課程概述"
# "課程目標"
# "課程要求"
# 欄位

course = course[["課程名稱", "課程流水號", "所屬系所", "授課教師", "上課星期", "上課節次", "課程概述", "課程目標", "課程要求"]]

In [32]:
print(course.shape)

(17343, 9)


In [33]:
# if "課程名稱" contain "專題討論"， drop the row
course = course[~course["課程名稱"].str.contains("專題討論")]

In [34]:
print(course.shape)

(16627, 9)


In [35]:
# if "課程概述" or "課程目標" or "課程要求" is empty, remove the row
course = course.dropna(subset=["課程概述", "課程目標", "課程要求"])

In [36]:
# see how many rows left
print(course.shape)

(8859, 9)


In [37]:
# save the data to a csv file
course.to_csv('course-v1.csv', index=False)

---

##### how to embedding data
- only 課程概述 + 課程名稱?
- 還是要加課程上課日期, 課程目標, etc


##### multi label firtering
- 我要找文字探勘和 sentence embedding 相關的課程， "而且想要禮拜三早上或是禮拜五的"
- 前面可以用 RAG + LLM 處理，後面那部分如何？

##### 
'課程名稱', '課程概述', '課程目標', '課程要求' 加起來超過 512 個 token 怎麼辦？

- 叫 LLM 抓重點？
- ~~休學~~
- 把「該門課程依照字數限制拆成兩個 ( or 多個 ) item」，如有 1700 token 的話，拆成四個 item ？